# Test in Centro Hospitalar Universitário de São João's (CHUSP) Dataset

### Using the models from **Run076** (IRF), **Run111** (SRF), and **Run114** (PED) in Binary Implementation and model from **Run058** in Multi-class

### Multi-class Model

#### Resized Images

In [ ]:
from test_model import test_model

test_model(
    fold_test=1,
    model_name="UNet",
    weights_name="Run058_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    dataset="chusj",
    device_name="GPU",
    batch_size=1,
    patch_type="vertical",
    save_images=True,
    resize_images=True,
    final_test=True
)

#### Original Images

In [ ]:
from test_model import test_model

test_model(
    fold_test=1,
    model_name="UNet",
    weights_name="Run058_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    dataset="chusj",
    device_name="GPU",
    batch_size=1,
    patch_type="vertical",
    save_images=True,
    resize_images=False,
    final_test=True
)

### Binary Models

#### Resized Images

In [ ]:
from test_model import test_model

test_model(
    fold_test=1,
    model_name="UNet3",
    weights_name="Run076111114_UNet3",
    number_of_channels=1,
    number_of_classes=2,
    dataset="chusj",
    device_name="GPU",
    batch_size=1,
    patch_type="vertical",
    save_images=True,
    resize_images=True,
    final_test=True,
    selected_models=[76,111,114],
    mode='highest_prob'
)

#### Original Images

In [ ]:
from test_model import test_model

test_model(
    fold_test=1,
    model_name="UNet3",
    weights_name="Run076111114_UNet3",
    number_of_channels=1,
    number_of_classes=2,
    dataset="chusj",
    device_name="GPU",
    batch_size=1,
    patch_type="vertical",
    save_images=True,
    resize_images=False,
    final_test=True,
    selected_models=[76,111,114],
    mode='highest_prob'
)